# MoneyPrinterPlus Notebook Workflow

This notebook provides a step-by-step workflow to generate a video from a text topic using the existing services of the MoneyPrinterPlus application.

## 1. Setup and Configuration

This cell imports necessary modules, loads the application's configuration, and creates a mock Streamlit session state. This session state is crucial because the application's services are designed to read their parameters from it.

In [ ]:
import os
import sys
import streamlit as st

# Add project root to Python path
project_root = os.path.abspath(os.path.join(os.getcwd()))
if project_root not in sys.path:
    sys.path.append(project_root)

# --- Mock Streamlit Session State ---
class MockSessionState(dict):
    def __getattr__(self, key):
        return self.get(key)

    def __setattr__(self, key, value):
        self[key] = value

# Replace the actual st.session_state with our mock object
st.session_state = MockSessionState()

# --- Load Application Configuration ---
from config.config import my_config
from main import (
    main_generate_video_content,
    main_generate_video_dubbing,
    main_generate_subtitle,
    main_get_video_resource
)
from services.video.video_service import VideoService
from services.captioning.captioning_service import add_subtitles
from tools.utils import get_must_session_option

print("Configuration and services loaded.")

## 2. Set Video Parameters

Define the topic and parameters for the video you want to create. These values are placed into the mock session state for the services to use.

In [ ]:
# --- User-defined parameters ---
st.session_state['video_subject'] = "The History of the Roman Empire"
st.session_state['video_language'] = 'en-US' # e.g., 'en-US', 'zh-CN'
st.session_state['video_length'] = '60' # Corresponds to character count

# --- Audio Configuration ---
# 'remote' for cloud services (Azure, etc.), 'local' for (ChatTTS, etc.)
st.session_state['audio_type'] = my_config.get('audio', {}).get('type', 'remote') 
st.session_state['audio_language'] = 'en-US'
st.session_state['audio_voice'] = 'en-US-AvaMultilingualNeural' # Example voice, change as needed
st.session_state['audio_speed'] = 'normal'

# --- Subtitle Configuration ---
st.session_state['enable_subtitles'] = True
st.session_state['recognition_audio_type'] = my_config.get('audio', {}).get('recognition_type', 'remote')
st.session_state['subtitle_font'] = 'STSong'
st.session_state['subtitle_font_size'] = 16
st.session_state['subtitle_color'] = '#FFFFFF'
st.session_state['subtitle_border_color'] = '#000000'
st.session_state['subtitle_border_width'] = 1
st.session_state['subtitle_position'] = 2 # Bottom center

# --- Video Configuration ---
st.session_state['video_fps'] = 25
st.session_state['video_size'] = '1080x1920' # Portrait
st.session_state['video_segment_min_length'] = 5
st.session_state['video_segment_max_length'] = 10
st.session_state['enable_background_music'] = True
st.session_state['background_music'] = os.path.join(project_root, 'bgmusic', 'default.mp3')
st.session_state['background_music_volume'] = 0.3
st.session_state['enable_video_transition_effect'] = True
st.session_state['video_transition_effect_duration'] = '1'
st.session_state['video_transition_effect_type'] = 'fade'
st.session_state['video_transition_effect_value'] = 'fade'

print(f"Parameters set for video topic: {st.session_state['video_subject']}")

## 3. Run the Video Generation Workflow

This cell executes the main functions in sequence to generate the video.

In [ ]:
try:
    # Step 1: Generate Script and Keywords
    print("[1/5] Generating video script and keywords...")
    main_generate_video_content()
    print(f"  - Script: {st.session_state.get('video_content')[:100]}...")
    print(f"  - Keywords: {st.session_state.get('video_keyword')}")

    # Step 2: Generate Audio from Script
    print("[2/5] Generating audio from script...")
    main_generate_video_dubbing()
    audio_file = st.session_state.get('audio_output_file')
    print(f"  - Audio file created at: {audio_file}")

    # Step 3: Download Source Videos
    print("[3/5] Downloading video resources...")
    main_get_video_resource()
    video_list = st.session_state.get('return_videos')
    print(f"  - Downloaded {len(video_list)} video clips.")

    # Step 4: Generate Subtitles
    print("[4/5] Generating subtitles...")
    main_generate_subtitle()
    subtitle_file = st.session_state.get('captioning_output')
    print(f"  - Subtitle file created at: {subtitle_file}")

    # Step 5: Assemble the Final Video
    print("[5/5] Assembling the final video...")
    video_service = VideoService(video_list, audio_file)
    print("  - Normalizing video clips...")
    video_service.normalize_video()
    print("  - Generating video with audio...")
    video_file_no_subs = video_service.generate_video_with_audio()
    
    if st.session_state.get("enable_subtitles"):
        print("  - Burning subtitles into video...")
        add_subtitles(
            video_file=video_file_no_subs, 
            subtitle_file=subtitle_file,
            font_name=st.session_state.get('subtitle_font'),
            font_size=st.session_state.get('subtitle_font_size'),
            primary_colour=st.session_state.get('subtitle_color'),
            outline_colour=st.session_state.get('subtitle_border_color'),
            outline=st.session_state.get('subtitle_border_width'),
            alignment=st.session_state.get('subtitle_position')
        )
    st.session_state["result_video_file"] = video_file_no_subs
    print("Workflow complete!")

except Exception as e:
    print(f"An error occurred: {e}")
    import traceback
    traceback.print_exc()

## 4. Display Result

The path to the final generated video is stored in the session state. Print the path to the final video file.

In [ ]:
final_video_path = st.session_state.get("result_video_file")
if final_video_path and os.path.exists(final_video_path):
    print(f"Successfully generated video:")
    print(final_video_path)
    # To display the video in a Jupyter environment, you can use:
    # from IPython.display import Video
    # Video(final_video_path, embed=True)
else:
    print("Video generation failed. Please check the logs above.")